In [1]:
!python setup.py install

  Cloning https://github.com/shahashka/SP-GIES to /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-k3jxy66h
  Running command git clone --filter=blob:none --quiet https://github.com/shahashka/SP-GIES /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-k3jxy66h
  Resolved https://github.com/shahashka/SP-GIES to commit 953fd745efd0c39b7bb67c9b2d5f0b15362884a3
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... - done


In [3]:
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.4 MB/s eta 0:00:00


In [8]:
!pip install ../CausalDiscoveryToolbox/

Processing /Users/shah38/Desktop/SP-GIES/CausalDiscoveryToolbox
  Preparing metadata (setup.py) ... - done
  Created wheel for cdt: filename=cdt-0.6.0-py3-none-any.whl size=921479 sha256=b433fb0a3a32f3d1188c5c624f739ac97dc9890e7d5f573a8a82f80184ca76be
  Stored in directory: /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-ephem-wheel-cache-l0671h_w/wheels/63/90/46/cebf55f777fa2087322d6997b55ee2f05cf89d22cad06fc7a0
Successfully built cdt
  Attempting uninstall: cdt
    Found existing installation: cdt 0.6.0
    Uninstalling cdt-0.6.0:
      Successfully uninstalled cdt-0.6.0


In [10]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 2.2 MB/s eta 0:00:00
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 2.5 MB/s eta 0:00:00


In [12]:
!pip install graphical_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 858.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.5 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)


In [14]:
!pip install causaldag

  Using cached causaldag-0.1a163-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.2/522.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 1.6 MB/s eta 0:00:00
  Using cached typing-3.7.4.3-py3-none-any.whl
  Using cached graphical_model_learning-0.1a8-py3-none-any.whl (62 kB)
  Using cached conditional_independence-0.1a6-py3-none-any.whl (23 kB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.4 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.1 MB/s eta 0:00:00
  Using cached progressbar2-4.2.0-

In [22]:
!pip install rpy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - \ | / - \ | / - \ | / - \ | / - \ done
  Preparing metadata (pyproject.toml) ... - \ | / - \ | / - \ | / - \ | done
  Using cached cffi-1.15.1-cp39-cp39-macosx_10_9_x86_64.whl (179 kB)
  Using cached tzlocal-4.3-py3-none-any.whl (20 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Created wheel for rpy2: filename=rpy2-3.5.11-cp39-cp39-macosx_10_9_x86_64.whl size=251303 sha256=945d206efe01e03488a163794f10fa320bc981c6e9521e2ab8e560317fdb2f4d
  Stored in directory: /Users/shah38/Library/Caches/pip/wheels/de/0d/32/6ea72f7349bf23f289f846826cd86880cb7ac8a08e3dfc66b1
Successfully built rpy2


In [32]:
from sp_gies import utils
from sp_gies import sp_gies as sp

SyntaxError: invalid syntax (2419340217.py, line 1)

In [41]:
(arcs,nodes,bias,var), data = utils.get_random_graph_data("scale_free", n=10, nsamples=10000, iv_samples=10, p=0.5, k=2)

In [42]:
print(data.head)

<bound method NDFrame.head of           G1        G2        G3         G4        G5        G6        G7   
0  -2.055289 -1.921275 -0.117713  -7.606389 -0.038671  1.630141 -0.360784  \
1  -5.764092 -1.747975 -3.162728  -9.251062 -3.022835 -6.075273 -1.546156   
2  -1.311946 -0.378803 -0.848973  -3.429204 -0.115179 -0.415790 -0.097351   
3  -6.263799 -1.953676 -3.994739 -13.663936 -3.402035 -6.413198 -1.347391   
4  -4.854423 -2.373262 -1.486435 -10.935424 -1.258019 -4.136905 -1.594045   
..       ...       ...       ...        ...       ...       ...       ...   
5  -1.387193  0.058825 -0.445174  -3.984471  1.291974  2.310300  0.129475   
6  -4.307012 -1.316742 -2.538545  -7.924621 -1.560417 -4.972337 -1.331786   
7   0.934764  0.162235  0.484399  -2.876811  0.401038  3.824387 -0.699011   
8   2.636336  0.715235  1.802051   3.102424  1.468067  8.414991  0.687464   
9   2.819422  0.983517  2.770674   3.363446  3.144328  6.324709  0.601532   

          G8        G9       G10  target  
0 

In [34]:
!mkdir test

In [43]:
adj_mat = sp.sp_gies(data, outdir="./test", skel=None, cupc=False)

In [44]:
G_true = utils.edge_to_dag(arcs)
G_sp_gies = utils.adj_to_dag(adj_mat, nodes)
print(utils.get_scores(["SP-GIES"], [G_sp_gies], G_true, get_sid=True))

SP-GIES 0.0 0.0 1.0
None
